In [72]:
import pandas as pd
import requests
from plyer import notification
import sqlite3

In [73]:
# URL base da API
base_url = "https://brasilapi.com.br/api/"

In [74]:
# Função para exibir um alerta de erro usando Plyer
def alerta(mensagem):
    notification.notify(
        title= "Erro na API BrasilAPI",
        message= mensagem,
        app_name= "Projeto Final",
    )

In [75]:
# Função para buscar dados da API
def buscar_dados(endpoint):
    url = f"{base_url}{endpoint}"
    response = requests.get(url)
    
    if response.status_code == 200:
        return response.json()
    else:
        mensagem = f"Erro ao buscar dados da API. Código de status: {response.status_code}"
        alerta(mensagem)  # Chama a função de alerta
        return None

In [76]:
#Criado a conexão ao banco de dados.
conn = sqlite3.connect('ProjetoFinalDB.db')
cursor = conn.cursor()

In [77]:
# Tabela 1: Lista de Taxas do Brasil
cursor.execute('''CREATE TABLE IF NOT EXISTS Taxas (
                    Nome TEXT,
                    Valor NUMERIC
                )''')

# Tabela 2: Lista de Pix do Brasil
cursor.execute('''CREATE TABLE IF NOT EXISTS Pixs (
                    ISPB NUMERIC,
                    Nome TEXT,
                    NomeReduzido TEXT,
                    Modalidade TEXT,
                    Tipo TEXT
                )''')

# Tabela 2: Lista dos CPTECs
cursor.execute('''CREATE TABLE IF NOT EXISTS CPTECS (
                    Nome TEXT,
                    Estado TEXT,
                    Id NUMERIC
                )''')



In [78]:
# Tabela 1: Lista de Taxas do Brasil
taxas = buscar_dados("taxas/v1")
if taxas:
    
    # Criar um DataFrame com os atributos
    nome = [taxa['nome'] for taxa in taxas]
    valor = [taxa['valor'] for taxa in taxas]

    df_taxas = pd.DataFrame({
        "Nome": nome,
        "Valor": valor,
    })
    
    df_taxas.fillna(0, inplace=True) #Tratamento de valores ausentes
    df_taxas['Valor'] = df_taxas['Valor'].astype(float) #Conversão de tipo de coluna

    print("Tabela 1: Lista de Taxas")
    print(df_taxas.head(5))  # Exibe as 5 primeiras linhas do DataFrame

Tabela 1: Lista de Taxas
    Nome  Valor
0  Selic  12.75
1    CDI  12.65
2   IPCA   4.61


In [79]:
# Tabela 2: Lista de PIX do Brasil
pixs = buscar_dados("pix/v1/participants")
if pixs:

    # Criar um DataFrame com os atributos
    ispb = [pix['ispb'] for pix in pixs]
    nome = [pix['nome'] for pix in pixs]
    nomeReduzido = [pix['nomeReduzido'] for pix in pixs]
    modalidade_participacao = [pix['modalidade_participacao'] for pix in pixs]
    tipo_participacao = [pix['tipo_participacao'] for pix in pixs]

    df_pixs = pd.DataFrame({
        "ISPB": ispb,
        "Nome": nome,
        "NomeReduzido": nomeReduzido,
        "Modalidade": modalidade_participacao,
        "Tipo": tipo_participacao
    })
    
    df_pixs.fillna(0, inplace=True) #Tratamento de valores ausentes
    df_pixs_stacked = df_pixs.set_index(["ISPB", "Nome", "NomeReduzido"]).stack().reset_index()
 
    print(df_pixs_stacked.head(5)) # Exibe as 5 primeiras linhas do DataFrame

       ISPB                                             Nome  \
0  00000000                               BCO DO BRASIL S.A.   
1  00000208                       BRB - BCO DE BRASILIA S.A.   
2  00068987                                CC ARACREDI LTDA.   
3  00075847  CC UNICRED CENTRO-SUL LTDA - UNICRED CENTRO-SUL   
4  00106180         CCC DOS EST DE MT, MS E MUN DE CACOAL/RO   

                                     Nome Reduzido Modalidade  Tipo  
0                               BCO DO BRASIL S.A.       PDCT  DRCT  
1                       BRB - BCO DE BRASILIA S.A.       PDCT  DRCT  
2                                CC ARACREDI LTDA.       PDCT  IDRT  
3  CC UNICRED CENTRO-SUL LTDA - UNICRED CENTRO-SUL       PDCT  IDRT  
4         CCC DOS EST DE MT, MS E MUN DE CACOAL/RO       PDCT  IDRT  


In [80]:
# Tabela 3: Lista IBGE
cptecs = buscar_dados("cptec/v1/cidade")
if cptecs:

    nome = [cptec['nome'] for cptec in cptecs]
    estado = [cptec['estado'] for cptec in cptecs]
    id = [cptec['id'] for cptec in cptecs]

    df_cptec = pd.DataFrame({
        'Nome': nome,
        'Estado': estado,
        'Id': id,
    })
    
    df_cptec.fillna(0, inplace=True) #Tratamento de valores ausentes
    estados_nordeste = ['AL', 'BH', 'CE', 'MA', 'PB', 'PE', 'PI', 'NA', 'SG'] # Lista de estados do Nordeste
    df_cptec_nordeste = df_cptec[df_cptec['Estado'].isin(estados_nordeste)]# Aplicar o filtro para pegar apenas os estados do Nordeste

    print(df_cptec_nordeste.head(5))

                        Nome Estado    Id
0               São Benedito     CE  4750
1  São Benedito do Rio Preto     MA  4751
2        São Benedito do Sul     PE  4752
3                  São Bento     MA  4753
4                  São Bento     PB  4754


In [81]:
#DataFrames salvos nas tabelas do banco de dados
if 'df_taxas' in locals():
    df_taxas.to_sql('Taxas', conn, if_exists='replace', index=False)

if 'df_pixs_stacked' in locals():
    df_pixs_stacked.to_sql('Pixs', conn, if_exists='replace', index=False)

if 'df_pixs_stacked' in locals():
    df_cptec_nordeste.to_sql('CPTECS', conn, if_exists='replace', index=False)

conn.commit()
conn.close()